In [36]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

from lightgbm import LGBMRegressor

# ============================================================
# K-League Track1 - LGBM Direct Regression (Wide Feature + CV)
# ------------------------------------------------------------
# ✅ 목표: end_x, end_y를 BIN 없이 "직접 회귀"로 예측
# ✅ 핵심: 마지막 K 이벤트를 wide feature로 펼친 뒤,
#         LGBMRegressor로 target_x, target_y 각각 학습
# ✅ 옵션: KFold로 CV(OOF) 점수 확인 후, 전체 학습 → Test 예측
#
# ⚠️ BASE_PATH 및 파일 경로는 반드시 그대로 유지
# ============================================================

# ----------------------
# 0. 설정 (경로/하이퍼파라미터)
# ----------------------
BASE_PATH = "open_track1/"
PATH_TRAIN = os.path.join(BASE_PATH, "train.csv")
PATH_TEST = os.path.join(BASE_PATH, "test.csv")
PATH_MATCH_INFO = os.path.join(BASE_PATH, "match_info.csv")
PATH_SAMPLE_SUB = os.path.join(BASE_PATH, "sample_submission.csv")

# 여러 K 값 비교 (30 이상)
K_LIST = [32, 35, 40, 45]  # 비교할 K 값들
K_BEST = None  # 최고 성능 K 값 (자동 설정됨)

# CV 설정
USE_CV = True
N_SPLITS = 5
SEED = 42

# LGBM 기본 파라미터 (너무 과튜닝하지 말고, 안정적으로 시작)
LGB_PARAMS = dict(
    n_estimators=5000,          # early stopping으로 실제 사용 개수 자동 결정
    learning_rate=0.03,
    num_leaves=64,
    max_depth=-1,
    min_child_samples=50,       # (=min_data_in_leaf 비슷)
    subsample=0.8,              # bagging_fraction
    subsample_freq=1,           # bagging_freq
    colsample_bytree=0.8,       # feature_fraction
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=SEED,
    n_jobs=-1
)

# ----------------------
# 1. 데이터 로드
# ----------------------
train = pd.read_csv(PATH_TRAIN)
test_index = pd.read_csv(PATH_TEST)
match_info = pd.read_csv(PATH_MATCH_INFO)  # 현재 코드에서는 직접 사용 안 함 (원본 유지)
sample_sub = pd.read_csv(PATH_SAMPLE_SUB)

# test는 episode별 csv로 따로 존재 → 합쳐서 events로 만들기
test_events_list = []
for _, row in test_index.iterrows():
    test_path = os.path.join(BASE_PATH, row["path"].lstrip("./"))
    df_ep = pd.read_csv(test_path)
    test_events_list.append(df_ep)

test_events = pd.concat(test_events_list, ignore_index=True)

train["is_train"] = 1
test_events["is_train"] = 0

events = pd.concat([train, test_events], ignore_index=True)

# ----------------------
# 2. 기본 정렬 + episode 내 인덱스
# ----------------------
events = events.sort_values(["game_episode", "time_seconds", "action_id"]).reset_index(drop=True)

events["event_idx"] = events.groupby("game_episode").cumcount()
events["n_events"] = events.groupby("game_episode")["event_idx"].transform("max") + 1
events["ep_idx_norm"] = events["event_idx"] / (events["n_events"] - 1).clip(lower=1)

# ----------------------
# 3. 시간/공간 feature
# ----------------------
events["prev_time"] = events.groupby("game_episode")["time_seconds"].shift(1)
events["dt"] = (events["time_seconds"] - events["prev_time"]).fillna(0.0)

events["dx"] = events["end_x"] - events["start_x"]
events["dy"] = events["end_y"] - events["start_y"]
events["dist"] = np.sqrt(events["dx"]**2 + events["dy"]**2)

# dt=0 보호
events["speed"] = events["dist"] / events["dt"].replace(0, 1e-3)

# zone / lane
events["x_zone"] = (events["start_x"] / (105/7)).astype(int).clip(0, 6)
events["lane"] = pd.cut(
    events["start_y"],
    bins=[0, 68/3, 2*68/3, 68],
    labels=[0, 1, 2],
    include_lowest=True
).astype(int)

# ----------------------
# 3-1. 코너 관련 피처 (Corner Features)
# ----------------------
events["angle_to_goal"] = np.arctan2(
    34 - events["start_y"],
    105 - events["start_x"]
) * 180 / np.pi

events["dist_corner_top"] = np.sqrt((105 - events["start_x"])**2 + (0 - events["start_y"])**2)
events["dist_corner_bottom"] = np.sqrt((105 - events["start_x"])**2 + (68 - events["start_y"])**2)
events["dist_to_nearest_corner"] = events[["dist_corner_top", "dist_corner_bottom"]].min(axis=1)

events["is_corner_area"] = ((events["start_x"] > 100) &
                            ((events["start_y"] < 5) | (events["start_y"] > 63))).astype(int)

events["angle_goal_x_corner"] = events["angle_to_goal"] * events["is_corner_area"]
events["dist_corner_x_angle"] = events["dist_to_nearest_corner"] * events["angle_to_goal"]

# ----------------------
# 3-2. 새로운 핵심 피처 3개 (고득점용)
# ----------------------
events["dist_to_sideline"] = events["start_y"].apply(lambda y: min(y, 68-y))

# 라디안 단위
events["angle_to_goal_center"] = np.arctan2(
    34 - events["start_y"],
    105 - events["start_x"]
)

events["time_pos_inter"] = events["ep_idx_norm"] * events["start_x"]

print("✓ 핵심 피처 3개 추가 완료 (dist_to_sideline, angle_to_goal_center, time_pos_inter)")

# ----------------------
# 4. 라벨 및 episode-level 메타 (train 전용)
# ----------------------
train_events = events[events["is_train"] == 1].copy()

last_events = (
    train_events
    .groupby("game_episode", as_index=False)
    .tail(1)
    .copy()
)

labels = last_events[["game_episode", "end_x", "end_y"]].rename(
    columns={"end_x": "target_x", "end_y": "target_y"}
)

ep_meta = last_events[["game_episode", "game_id", "team_id", "is_home", "period_id", "time_seconds"]].copy()
ep_meta = ep_meta.rename(columns={"team_id": "final_team_id"})

ep_meta["game_clock_min"] = np.where(
    ep_meta["period_id"] == 1,
    ep_meta["time_seconds"] / 60.0,
    45.0 + ep_meta["time_seconds"] / 60.0
)

# ----------------------
# 5. 공격 팀 플래그 (final_team vs 상대)
# ----------------------
events = events.merge(
    ep_meta[["game_episode", "final_team_id"]],
    on="game_episode",
    how="left"
)
events["is_final_team"] = (events["team_id"] == events["final_team_id"]).astype(int)

# ----------------------
# 6. 입력용 events에서 마지막 이벤트 타깃 정보 가리기
# ----------------------
events["last_idx"] = events.groupby("game_episode")["event_idx"].transform("max")
events["is_last"] = (events["event_idx"] == events["last_idx"]).astype(int)

mask_last = events["is_last"] == 1
for col in ["end_x", "end_y", "dx", "dy", "dist", "speed"]:
    events.loc[mask_last, col] = np.nan

# ----------------------
# 7. 카테고리 인코딩 (type_name, result_name, team_id 등)
# ----------------------
events["type_name"] = events["type_name"].fillna("__NA_TYPE__")
events["result_name"] = events["result_name"].fillna("__NA_RES__")

le_type = LabelEncoder()
le_res = LabelEncoder()

events["type_id"] = le_type.fit_transform(events["type_name"])
events["res_id"] = le_res.fit_transform(events["result_name"])

if events["team_id"].dtype == "object":
    le_team = LabelEncoder()
    events["team_id_enc"] = le_team.fit_transform(events["team_id"])
else:
    events["team_id_enc"] = events["team_id"].astype(int)

# ----------------------
# 8. 마지막 K 이벤트만 사용 (lastK)
# ----------------------
events["rev_idx"] = events.groupby("game_episode")["event_idx"].transform(
    lambda s: s.max() - s
)
lastK = events[events["rev_idx"] < K].copy()

def assign_pos_in_K(df):
    df = df.sort_values("event_idx")  # 오래된 → 최근
    L = len(df)
    df = df.copy()
    df["pos_in_K"] = np.arange(K - L, K)
    return df

lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)

# ----------------------
# 9. wide feature pivot
# ----------------------
num_cols = [
    "start_x", "start_y",
    "end_x", "end_y",
    "dx", "dy", "dist", "speed",
    "dt",
    "ep_idx_norm",
    "x_zone", "lane",
    "is_final_team",
    # 코너 관련
    "angle_to_goal",
    "dist_to_nearest_corner",
    "is_corner_area",
    "angle_goal_x_corner",
    "dist_corner_x_angle",
    # 핵심 3개
    "dist_to_sideline",
    "angle_to_goal_center",
    "time_pos_inter",
]

cat_cols = [
    "type_id",
    "res_id",
    "team_id_enc",
    "is_home",
    "period_id",
    "is_last",
]

feature_cols = num_cols + cat_cols
wide = lastK[["game_episode", "pos_in_K"] + feature_cols].copy()

wide_num = wide.pivot_table(index="game_episode", columns="pos_in_K", values=num_cols, aggfunc="first")
wide_cat = wide.pivot_table(index="game_episode", columns="pos_in_K", values=cat_cols, aggfunc="first")

wide_num.columns = [f"{c}_{int(pos)}" for (c, pos) in wide_num.columns]
wide_cat.columns = [f"{c}_{int(pos)}" for (c, pos) in wide_cat.columns]

X = pd.concat([wide_num, wide_cat], axis=1).reset_index()

# episode-level 메타 붙이기
X = X.merge(
    ep_meta[["game_episode", "game_id", "game_clock_min", "final_team_id", "is_home", "period_id"]],
    on="game_episode",
    how="left"
)

# train 라벨 붙이기
X = X.merge(labels, on="game_episode", how="left")

# ----------------------
# 10. train/test 분리
# ----------------------
train_mask = X["game_episode"].isin(labels["game_episode"])
X_train = X[train_mask].copy()
X_test = X[~train_mask].copy()

y_train_x = X_train["target_x"].astype(float).values
y_train_y = X_train["target_y"].astype(float).values

drop_cols = ["game_episode", "game_id", "target_x", "target_y"]

X_train_feat = X_train.drop(columns=drop_cols)
X_test_feat = X_test.drop(columns=[c for c in drop_cols if c in X_test.columns])

# 결측 처리
X_train_feat = X_train_feat.fillna(0)
X_test_feat = X_test_feat.fillna(0)

# is_home 컬럼을 숫자로 변환 (LightGBM 호환성)
for col in X_train_feat.columns:
    if "is_home" in col and X_train_feat[col].dtype == "object":
        X_train_feat[col] = pd.to_numeric(X_train_feat[col], errors="coerce").fillna(0).astype(int)
        if col in X_test_feat.columns:
            X_test_feat[col] = pd.to_numeric(X_test_feat[col], errors="coerce").fillna(0).astype(int)
    
    # period_id 컬럼도 확인
    if "period_id" in col and X_train_feat[col].dtype == "object":
        X_train_feat[col] = pd.to_numeric(X_train_feat[col], errors="coerce").fillna(1).astype(int)
        if col in X_test_feat.columns:
            X_test_feat[col] = pd.to_numeric(X_test_feat[col], errors="coerce").fillna(1).astype(int)

# ----------------------
# 8. Wide feature 생성
# ----------------------
events["rev_idx"] = events.groupby("game_episode")["event_idx"].transform(
    lambda s: s.max() - s
)
lastK = events[events["rev_idx"] < K].copy()

def assign_pos_in_K(df):
    df = df.sort_values("event_idx")
    L = len(df)
    df = df.copy()
    df["pos_in_K"] = np.arange(K - L, K)
    return df

lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)

feature_cols = num_cols + cat_cols
wide = lastK[["game_episode", "pos_in_K"] + feature_cols].copy()

wide_num = wide.pivot_table(index="game_episode", columns="pos_in_K", values=num_cols, aggfunc="first")
wide_cat = wide.pivot_table(index="game_episode", columns="pos_in_K", values=cat_cols, aggfunc="first")

wide_num.columns = [f"{c}_{int(pos)}" for (c, pos) in wide_num.columns]
wide_cat.columns = [f"{c}_{int(pos)}" for (c, pos) in wide_cat.columns]

X = pd.concat([wide_num, wide_cat], axis=1).reset_index()

# episode-level 메타 붙이기
X = X.merge(
    ep_meta[["game_episode", "game_id", "game_clock_min", "final_team_id", "is_home", "period_id"]],
    on="game_episode",
    how="left"
)

# train 라벨 붙이기
X = X.merge(labels, on="game_episode", how="left")

# ----------------------
# 9. train/test 분리
# ----------------------
train_mask = X["game_episode"].isin(labels["game_episode"])
X_train = X[train_mask].copy()
X_test = X[~train_mask].copy()

y_train_x = X_train["target_x"].astype(float).values
y_train_y = X_train["target_y"].astype(float).values

drop_cols = ["game_episode", "game_id", "target_x", "target_y"]

X_train_feat = X_train.drop(columns=drop_cols)
X_test_feat = X_test.drop(columns=[c for c in drop_cols if c in X_test.columns])

# 결측 처리
X_train_feat = X_train_feat.fillna(0)
X_test_feat = X_test_feat.fillna(0)

# is_home 컬럼을 숫자로 변환 (LightGBM 호환성)
for col in X_train_feat.columns:
    if "is_home" in col and X_train_feat[col].dtype == "object":
        X_train_feat[col] = pd.to_numeric(X_train_feat[col], errors="coerce").fillna(0).astype(int)
        if col in X_test_feat.columns:
            X_test_feat[col] = pd.to_numeric(X_test_feat[col], errors="coerce").fillna(0).astype(int)
    
    # period_id 컬럼도 확인
    if "period_id" in col and X_train_feat[col].dtype == "object":
        X_train_feat[col] = pd.to_numeric(X_train_feat[col], errors="coerce").fillna(1).astype(int)
        if col in X_test_feat.columns:
            X_test_feat[col] = pd.to_numeric(X_test_feat[col], errors="coerce").fillna(1).astype(int)

# ----------------------
# 10. CV로 OOF 점수 확인
# ----------------------
def euclidean_mean(y_true_x, y_true_y, y_pred_x, y_pred_y):
    dx = y_true_x - y_pred_x
    dy = y_true_y - y_pred_y
    return np.mean(np.sqrt(dx*dx + dy*dy))

if USE_CV:
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    fold_scores = []
    
    print("=" * 80)
    print(f"📌 LGBM CV 시작: n_splits={N_SPLITS}, K={K}")
    print("=" * 80)
    
    for fold, (tr_idx, va_idx) in enumerate(kf.split(X_train_feat), 1):
        X_tr, X_va = X_train_feat.iloc[tr_idx], X_train_feat.iloc[va_idx]
        y_tr_x, y_va_x = y_train_x[tr_idx], y_train_x[va_idx]
        y_tr_y, y_va_y = y_train_y[tr_idx], y_train_y[va_idx]
        
        # X 모델
        model_x = LGBMRegressor(**LGB_PARAMS)
        model_x.fit(
            X_tr, y_tr_x,
            eval_set=[(X_va, y_va_x)],
            eval_metric="rmse",
        )
        
        # Y 모델
        model_y = LGBMRegressor(**LGB_PARAMS)
        model_y.fit(
            X_tr, y_tr_y,
            eval_set=[(X_va, y_va_y)],
            eval_metric="rmse",
        )
        
        pred_va_x = model_x.predict(X_va)
        pred_va_y = model_y.predict(X_va)
        
        # 필드 범위로 클립
        pred_va_x = np.clip(pred_va_x, 0, 105)
        pred_va_y = np.clip(pred_va_y, 0, 68)
        
        fold_euc = euclidean_mean(y_va_x, y_va_y, pred_va_x, pred_va_y)
        fold_scores.append(fold_euc)
        
        print(f"[Fold {fold}] mean Euclidean = {fold_euc:.6f}")
    
    cv_mean = float(np.mean(fold_scores))
    cv_std = float(np.std(fold_scores))
    
    print("=" * 80)
    print(f"✅ CV 완료 | mean Euclidean = {cv_mean:.6f} ± {cv_std:.6f}")
    print("=" * 80)

# ----------------------
# 12. 최종 학습(전체 train) + test 예측
# ----------------------
print("=" * 80)
print("📌 전체 Train으로 최종 모델 학습 후 Test 예측 시작...")
print("=" * 80)

final_model_x = LGBMRegressor(**LGB_PARAMS)
final_model_x.fit(X_train_feat, y_train_x)

final_model_y = LGBMRegressor(**LGB_PARAMS)
final_model_y.fit(X_train_feat, y_train_y)

pred_x = final_model_x.predict(X_test_feat)
pred_y = final_model_y.predict(X_test_feat)

pred_x = np.clip(pred_x, 0, 105)
pred_y = np.clip(pred_y, 0, 68)

# ----------------------
# 13. submission 생성 (원본 방식 유지)
# ----------------------
sub = sample_sub.copy()

pred_df = X_test[["game_episode"]].copy()
pred_df["end_x"] = pred_x
pred_df["end_y"] = pred_y

sub = sub.drop(columns=["end_x", "end_y"], errors="ignore")
sub = sub.merge(pred_df, on="game_episode", how="left")

submission_filename = f"submission_lgbm_direct_k{K}.csv"
sub.to_csv(submission_filename, index=False)

print(f"✅ Saved: {submission_filename}")
print("🎉 완료!")


✓ 핵심 피처 3개 추가 완료 (dist_to_sideline, angle_to_goal_center, time_pos_inter)


/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:220: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:291: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train_feat = X_train_feat.fillna(0)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:292: FutureWarning: Downcasting ob

📊 여러 K 값 비교 시작: [32, 35, 40, 45]

🔄 K=32 처리 중...


/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:342: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:379: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train_feat = X_train_feat.fillna(0)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:380: FutureWarning: Downcasting ob

📌 LGBM CV 시작: n_splits=5, K=32
[Fold 1] mean Euclidean = 14.667537
[Fold 2] mean Euclidean = 14.796664
[Fold 3] mean Euclidean = 14.588967
[Fold 4] mean Euclidean = 14.262552
[Fold 5] mean Euclidean = 14.656135
✅ K=32 CV 완료 | mean Euclidean = 14.594371 ± 0.179011

🔄 K=35 처리 중...


/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:342: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:379: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train_feat = X_train_feat.fillna(0)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:380: FutureWarning: Downcasting ob

📌 LGBM CV 시작: n_splits=5, K=35
[Fold 1] mean Euclidean = 14.683465
[Fold 2] mean Euclidean = 14.799056
[Fold 3] mean Euclidean = 14.529432
[Fold 4] mean Euclidean = 14.319081
[Fold 5] mean Euclidean = 14.694736
✅ K=35 CV 완료 | mean Euclidean = 14.605154 ± 0.166929

🔄 K=40 처리 중...


/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:342: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:379: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train_feat = X_train_feat.fillna(0)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/1529377035.py:380: FutureWarning: Downcasting ob

📌 LGBM CV 시작: n_splits=5, K=40
[Fold 1] mean Euclidean = 14.751481
[Fold 2] mean Euclidean = 14.781937
[Fold 3] mean Euclidean = 14.658883


KeyboardInterrupt: 